# Referral Priority Contacts
Use this view to focus on tagged contacts with the highest referral potential.

In [ ]:
from pathlib import Path
import pandas as pd
import yaml
from IPython.display import display

search_root = Path.cwd()
config_path = None
for candidate in [search_root, *search_root.parents]:
    maybe = candidate / 'config.yaml'
    if maybe.exists():
        config_path = maybe
        project_root = candidate
        break
if config_path is None:
    raise FileNotFoundError("Unable to locate config.yaml from current working directory.")

cfg = yaml.safe_load(config_path.read_text()) or {}
outputs_dir = (project_root / cfg.get('outputs', {}).get('dir', 'output')).resolve()
if not outputs_dir.exists():
    raise FileNotFoundError(f"Outputs directory not found: {outputs_dir}")

contacts_path = outputs_dir / 'consolidated_contacts.csv'
confidence_path = outputs_dir / 'confidence_report.csv'
referral_path = outputs_dir / 'referral_targets.csv'
tagged_path = outputs_dir / 'tagged_contacts.csv'

if referral_path.exists():
    df = pd.read_csv(referral_path, dtype=str, keep_default_na=False)
else:
    df = pd.read_csv(tagged_path, dtype=str, keep_default_na=False)

if 'referral_priority_score' in df.columns:
    df['referral_priority_score'] = pd.to_numeric(df['referral_priority_score'], errors='coerce')
if 'confidence_score' in df.columns:
    df['confidence_score'] = pd.to_numeric(df['confidence_score'], errors='coerce')

columns = [
    'contact_id',
    'full_name',
    'company',
    'title',
    'relationship_category',
    'referral_priority_score',
    'confidence_score',
    'confidence_bucket',
    'emails',
    'invalid_emails',
    'phones',
    'non_standard_phones',
]
display_cols = [col for col in columns if col in df.columns]
summary = df[display_cols]
sort_cols = [col for col in ['referral_priority_score', 'confidence_score'] if col in summary.columns]
if sort_cols:
    summary = summary.sort_values(sort_cols, ascending=[False] * len(sort_cols))
display(summary.head(25))
